# Linux的信号机制  

## signal基本概念
软中断信号（signal，又简称为信号）用来通知进程发生了异步事件。**进程之间可以互相通过系统调用kill命令发送软中断信号给特定的进程，进程收到信号后使用siganl函数作出相应的处理。**内核也可以因为内部事件而给进程发送信号，通知进程发生了某个事件。注意，信号只是用来通知某进程发生了什么事件，并不给该进程传递任何数据。  

**进程收到信号后有三种处理方式**  
1. 忽略某个信号，不进行任何处理(`SIG_IGN`)。
2. 使用系统的默认值进行操作（大部分信号的默认操作是终止进程, `SIG_DFL`）。
3. 指定处理函数，进行处理（`signal(x,y)`方法）。  

**signal()函数**  

```C++ 
//singal函数需要的头文件
#include <signal.h>
// 定义函数：
void (*signal(int signum, void(* handler)(int)))(int);  

```  
上述声明格式比较复杂，如果不清楚如何使用，也可以通过下面这种类型定义的格式来使用。
```C++
//（POSIX的定义）：
typedef void (*sighandler_t)(int);
sighandler_t signal(int signum, sighandler_t handler);
```
但这种格式在不同的系统中有不同的类型定义，所以要使用这种格式，最好还是参考一下联机手册。 

(1)**signum是信号的编号，是linux系统定义的宏定义**，输入kill -l可以查看全部的信号：
>  1) SIGHUP     2) SIGINT     3) SIGQUIT     4) SIGILL     5) SIGTRAP  
 6) SIGABRT     7) SIGBUS     8) SIGFPE     9) SIGKILL    10) SIGUSR1  
 11) SIGSEGV    12) SIGUSR2    13) SIGPIPE    14) SIGALRM    15) SIGTERM  
 16) SIGSTKFLT    17) SIGCHLD    18) SIGCONT    19) SIGSTOP    20) SIGTSTP  
 21) SIGTTIN    22) SIGTTOU    23) SIGURG    24) SIGXCPU    25) SIGXFSZ  
 26) SIGVTALRM    27) SIGPROF    28) SIGWINCH    29) SIGIO    30) SIGPWR  
 31) SIGSYS    34) SIGRTMIN    35) SIGRTMIN+1    36) SIGRTMIN+2    37) SIGRTMIN+3  
 38) SIGRTMIN+4    39) SIGRTMIN+5    40) SIGRTMIN+6    41) SIGRTMIN+7    42) SIGRTMIN+8  
 43) SIGRTMIN+9    44) SIGRTMIN+10    45) SIGRTMIN+11    46) SIGRTMIN+12    47) SIGRTMIN+13  
 48) SIGRTMIN+14    49) SIGRTMIN+15    50) SIGRTMAX-14    51) SIGRTMAX-13    52) SIGRTMAX-12  
 53) SIGRTMAX-11    54) SIGRTMAX-10    55) SIGRTMAX-9    56) SIGRTMAX-8    57) SIGRTMAX-7  
 58) SIGRTMAX-6    59) SIGRTMAX-5    
 
 (2)**handler是处理函数，或者是SIG_IGN,SIG_DFL**

## 示例  
进程收到信号后有三种处理方法，示例如下。
SIGINT信号是一个中断信号，如ctrl+c可以产生一个中断信号，一般由用户生成此类信号。  
```C++
//signal-funtion-test.cpp

#include <iostream>
#include <stdio.h>
#include <signal.h>
#include <unistd.h>

#include "signal-function-test.h"

//1.收到信号，忽略信号
void my_ignore_signal_main(){
    //ctrl + c to stop the program doesn`t work!
    signal(SIGINT, SIG_IGN);
    while (1){
        printf("ignore SIGINT signal! Try `ctrl + c`\n");
        sleep(1);
    }
}

//2.收到信号，使用默认操作
void my_default_signal_main(){
    signal(SIGINT, SIG_DFL);
    while (1){
        printf("Default SIGINT signal! Try `ctrl + c`\n");
        sleep(1);
    }
}

//3.收到信号，自定义操作
void my_defined_handler(int signum){
    printf("my_defined_handler_function. do something to handle the signal!");
}

void my_defined_signal_main(){
    signal(SIGINT, my_defined_handler);
    while (1){
        printf("Default SIGINT signal! Try `ctrl + c`\n");
        sleep(1);
    }
}

```

> 注：在控制台使用gcc命令编译时，会报错：
```C++
signal-function-test.cpp:(.text+0x75): undefined reference to `std::ios_base::Init::Init()'
signal-function-test.cpp:(.text+0x84): undefined reference to `std::ios_base::Init::~Init()'
```  
解决方法： “ you need to add -lstdc++, or use 'g++' rather than 'gcc' as your driver program.”  
这里涉及到**gcc和g++的区别**：
 1. gcc和g++都是GNU(组织)的一个编译器。
 2. 后缀名为.c的程序和.cpp的程序g++都会当成是c++的源程序来处理。而gcc不然，gcc会把.c的程序处理成c程序。  
 3. 对于.cpp的程序，编译可以用gcc/g++，而链接可以用g++或者gcc -lstdc++。  
 （由于我写的是.cpp文件，且使用gcc命令编译，在链接时需要添加-lstdc++参数！） 

**相关函数**  
> sigaction  
  kill  
  raise
  
**参考博客**：
[Linux 信号signal处理机制](http://www.cnblogs.com/taobataoma/archive/2007/08/30/875743.html)
